# Customer Segmentation using Spark
Customer segmentation is a marketing technique companies use to identify and group users who display similar characteristics. In this project, I am using Spark with Python i-e PySpark for building the customer segmentation model

## Dataset Used:
ecommerce-dataset

### Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

### 1. Creating a SparkSession
A SparkSession is an entry point into all functionality in Spark, and is required if you want to build a dataframe in PySpark

In [2]:
spark = SparkSession.builder.appName("Customer Segmentation Spark").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()


Using the codes above, we built a spark session and set a name for the application. Then, the data was cached in off-heap memory to avoid storing it directly on disk, and the amount of memory was manually specified.

### 2. Creating a DataFrame using Spark

In [3]:
df = spark.read.csv('ecommerce_data.csv',header=True,escape="\"")

Note that we defined an escape character to avoid commas in the .csv file when parsing.

In [4]:
df.show(5,0)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
+---------+---------+-----------------------------------

In [5]:
columns = df.columns 
columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

The dataframe consists of 8 variables:  
InvoiceNo: The unique identifier of each customer invoice.  
StockCode: The unique identifier of each item in stock.  
Description: The item purchased by the customer.  
Quantity: The number of each item purchased by a customer in a single invoice.  
InvoiceDate: The purchase date.  
UnitPrice: Price of one unit of each item.  
CustomerID: Unique identifier assigned to each user.  
Country: The country from where the purchase was made  

### 3. Exploratory Data Analysis

Counting the total rows in dataset

In [12]:
df.count()

541909

Counting the number of unique customers present in dataframe

In [13]:
df.select('CustomerID').distinct().count()

4373

Finding the country from where the most purchases are made using groupby

In [15]:
df.head(5)

[Row(InvoiceNo='536365', StockCode='85123A', Description='WHITE HANGING HEART T-LIGHT HOLDER', Quantity='6', InvoiceDate='12/1/2010 8:26', UnitPrice='2.55', CustomerID='17850', Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='71053', Description='WHITE METAL LANTERN', Quantity='6', InvoiceDate='12/1/2010 8:26', UnitPrice='3.39', CustomerID='17850', Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='84406B', Description='CREAM CUPID HEARTS COAT HANGER', Quantity='8', InvoiceDate='12/1/2010 8:26', UnitPrice='2.75', CustomerID='17850', Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='84029G', Description='KNITTED UNION FLAG HOT WATER BOTTLE', Quantity='6', InvoiceDate='12/1/2010 8:26', UnitPrice='3.39', CustomerID='17850', Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='84029E', Description='RED WOOLLY HOTTIE WHITE HEART.', Quantity='6', InvoiceDate='12/1/2010 8:26', UnitPrice='3.39', CustomerID='17850', Country='United Kingdom')]

In [14]:
df.tail(5)

[Row(InvoiceNo='581587', StockCode='22613', Description='PACK OF 20 SPACEBOY NAPKINS', Quantity='12', InvoiceDate='12/9/2011 12:50', UnitPrice='0.85', CustomerID='12680', Country='France'),
 Row(InvoiceNo='581587', StockCode='22899', Description="CHILDREN'S APRON DOLLY GIRL ", Quantity='6', InvoiceDate='12/9/2011 12:50', UnitPrice='2.1', CustomerID='12680', Country='France'),
 Row(InvoiceNo='581587', StockCode='23254', Description='CHILDRENS CUTLERY DOLLY GIRL ', Quantity='4', InvoiceDate='12/9/2011 12:50', UnitPrice='4.15', CustomerID='12680', Country='France'),
 Row(InvoiceNo='581587', StockCode='23255', Description='CHILDRENS CUTLERY CIRCUS PARADE', Quantity='4', InvoiceDate='12/9/2011 12:50', UnitPrice='4.15', CustomerID='12680', Country='France'),
 Row(InvoiceNo='581587', StockCode='22138', Description='BAKING SET 9 PIECE RETROSPOT ', Quantity='3', InvoiceDate='12/9/2011 12:50', UnitPrice='4.95', CustomerID='12680', Country='France')]

In [18]:
df_grouped_country = df.groupBy('Country').agg(countDistinct('CustomerID').alias('country_count')).orderBy(desc('country_count')).show()

+---------------+-------------+
|        Country|country_count|
+---------------+-------------+
| United Kingdom|         3950|
|        Germany|           95|
|         France|           87|
|          Spain|           31|
|        Belgium|           25|
|    Switzerland|           21|
|       Portugal|           19|
|          Italy|           15|
|        Finland|           12|
|        Austria|           11|
|         Norway|           10|
|        Denmark|            9|
|Channel Islands|            9|
|      Australia|            9|
|    Netherlands|            9|
|         Sweden|            8|
|         Cyprus|            8|
|          Japan|            8|
|         Poland|            6|
|         Greece|            4|
+---------------+-------------+
only showing top 20 rows



From this table it can be seen that almost all the purchases on platform were made in UK

Finding the date of most recent purchase by customer in e-commerce platform

In [22]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
df = df.withColumn('date',to_timestamp("InvoiceDate", 'dd/MM/yy HH:mm'))
df.select(max("date")).show()

+-------------------+
|          max(date)|
+-------------------+
|2011-12-10 17:19:00|
+-------------------+



In [23]:
df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|               date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-01-12 08:26:00|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12 08:26:00|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-01-12 08:26:00|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12 08:26:00|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12 08:26:00|
|   536365|    2

New column date was added to this dataframe based on InvoiceDate column which has date in proper format

Finding the earliest purchase made by customer on e-commerce platform

In [24]:
df.select(min('date')).show()

+-------------------+
|          min(date)|
+-------------------+
|2010-01-12 08:26:00|
+-------------------+

